In [2]:
import pandas as pd
import numpy as np
import os
import torch

In [3]:
nmt_df = pd.read_csv("data/nmt/nmt_full_seq.csv")
nmt_df = nmt_df.drop(columns=["38.1", "25.1", "178.1", "216.1"])

nmt_positions_to_embed =nmt_df.columns[(np.where(nmt_df.columns == "1")[0][0]):(np.where(nmt_df.columns == "272")[0][0]+1)].to_list()


In [30]:
import subprocess

def run_python_script(args, cwd=None):
    """
    Executes a Python script with the given arguments list in the specified directory.
    Prints STDOUT in real time.
    The first element should be the script path, followed by its arguments.
    Example: ["other_script.py", "--foo", "bar"]
    """
    cmd = ["python"] + args
    print("Running command:", " ".join(cmd))
    process = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True,
        cwd=cwd
    )

    while True:
        stdout_line = process.stdout.readline()
        if stdout_line:
            print(stdout_line, end="", flush=True)
        elif process.poll() is not None:
            break

    for line in process.stdout:
        print(line, end="", flush=True)

    stderr = process.stderr.read()
    if stderr:
        print("\n--- STDERR ---\n", stderr)

    process.stdout.close()
    process.stderr.close()
    return process


args = ["../code/embedding_engine.py",
        "--action", "embed_parallel",
        "--dataset_file", "%s/data/nmt/nmt_full_seq.csv" % os.getcwd(),
        #"--n_workers", "5",
        #"--execute_after_parallel_generation", "false",
        "--model", "esm2_t12_35M_UR50D",
        "--evaluation_path", "/%s/results/nmt_embeddings/esm_35m" % os.getcwd(),
        "--sequence_colname", "seq",
        "--label_column_name", "activity",
        "--job_executing_type", "lsf",
        "--positions_to_embed"] + nmt_positions_to_embed





In [4]:
run_python_script(args, cwd=os.path.abspath("../code"))

NameError: name 'run_python_script' is not defined

In [37]:
# Execute workers after jobs parallel generation

jobs_path = "/home/labs/fleishman/itayta/new_fitness_repo/fitness_learning/notebooks/results/nmt_embeddings/esm_35m/jobs_20251216_190708_parallel"
args = ["../code/embedding_engine.py",
        "--action", "execute_workers",
        "--job_executing_type", "lsf",
        "--jobs_path", "%s" % jobs_path,
        "--n_workers", "1"]

run_python_script(args, cwd=os.path.abspath("../code"))


# In case something fails!
# args = ["../code/embedding_engine.py",
#         "--action", "execute_workers",
#         "--job_executing_type", "lsf",
#         "--jobs_path", ##### 
#         "--specific_job_i", "1046",
#         "--n_workers", "10"]

# run_python_script(args, cwd=os.path.abspath("../code")) 

Running command: python ../code/embedding_engine.py --action execute_workers --job_executing_type lsf --jobs_path /home/labs/fleishman/itayta/new_fitness_repo/fitness_learning/notebooks/results/nmt_embeddings/esm_35m/jobs_20251216_190708_parallel --n_workers 1
[INFO] Initializing PLM with current working directory: /home/labs/fleishman/itayta/new_fitness_repo/fitness_learning
[INFO] Created job execution library at: /home/labs/fleishman/itayta/new_fitness_repo/fitness_learning/notebooks/results/nmt_embeddings/esm_35m/jobs_20251216_190708_parallel/job_execution_20251216_191215
Submitting job 1 via:  bsub -n 2 -gpu num=1:gmem=24G:aff=yes -R same[gpumodel] -R rusage[mem=64GB] -R span[ptile=2] -e /home/labs/fleishman/itayta/new_fitness_repo/fitness_learning/notebooks/results/nmt_embeddings/esm_35m/jobs_20251216_190708_parallel/job_execution_20251216_191215/job_outputs/err_file_worker_73339708 -o /home/labs/fleishman/itayta/new_fitness_repo/fitness_learning/notebooks/results/nmt_embeddings/

<Popen: returncode: 0 args: ['python', '../code/embedding_engine.py', '--act...>

In [38]:
args = ["../code/embedding_engine.py",
        "--action", "check_on_jobs",
        "--jobs_path", "%s" % jobs_path]

run_python_script(args, cwd=os.path.abspath("../code"))

Running command: python ../code/embedding_engine.py --action check_on_jobs --jobs_path /home/labs/fleishman/itayta/new_fitness_repo/fitness_learning/notebooks/results/nmt_embeddings/esm_35m/jobs_20251216_190708_parallel
[INFO] Initializing PLM with current working directory: /home/labs/fleishman/itayta/new_fitness_repo/fitness_learning
[INFO] Checking on jobs in: /home/labs/fleishman/itayta/new_fitness_repo/fitness_learning/notebooks/results/nmt_embeddings/esm_35m/jobs_20251216_190708_parallel
[INFO] Args: {'average_embeddings': False, 'positions_to_embed': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 1

<Popen: returncode: 0 args: ['python', '../code/embedding_engine.py', '--act...>

In [5]:
embeddings = torch.load("./results/nmt_embeddings/esm_35m/evaluations/0_238/train/embeddings.pt")
embeddings.shape

/tmp/ipykernel_2456110/1919270262.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  embeddings = torch.load("./results/nmt_embeddings/esm_35m/evaluations/0_238/train/embed

torch.Size([238, 267, 480])

In [6]:
si = np.where(nmt_df.columns == "1")[0][0]
ei = np.where(nmt_df.columns == "272")[0][0]+1

wt_ref = nmt_df.iloc[0,si:ei]

mutations_tensor =\
[torch.tensor([int(x1 != x2) for x1, x2 in zip(wt_ref, nmt_df.iloc[i,si:ei])], dtype=torch.int) for i in range(1, nmt_df.shape[0])]

mutations_matrix = torch.stack(mutations_tensor, dim=0)
mutations_matrix.shape

no_wt = embeddings[1:,:]



In [7]:
# Sanity
assert(sum(nmt_df.iloc[1:,:]["num_muts"].to_numpy() == mutations_matrix.sum(dim=1).numpy()) == nmt_df.shape[0] - 1)

In [8]:
mutated_positions_embedding = embeddings[:,torch.where(mutations_matrix.sum(dim=0) != 0)[0],:]

In [9]:
torch.save(mutated_positions_embedding, "data/nmt/embeddings/esm_35m/embeddings.pt")

indices = torch.load("./results/nmt_embeddings/esm_35m/evaluations/0_238/train/indices.pt")
torch.save(indices, "data/nmt/embeddings/esm_35m/indices.pt")

y_values = torch.load("./results/nmt_embeddings/esm_35m/evaluations/0_238/train/y_value.pt")
torch.save(y_values, "data/nmt/embeddings/esm_35m/y_values.pt")

/tmp/ipykernel_2456110/2062947111.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  indices = torch.load("./results/nmt_embeddings/esm_35m/evaluations/0_238/train/indices.